# DSA 2040A – ETL Midterm Exam
## Data Transformation Phase

This notebook handles data transformation of the extracted full and incremental datasets. At least 4 meaningful transformations are applied to prepare the data for analysis.

In [1]:
# Section 1: Setup and Data Loading

import pandas as pd
import os

# Load extracted datasets from the data/ folder
df_raw = pd.read_csv("data/raw_data.csv")
df_incremental = pd.read_csv("data/incremental_data.csv")

# Preview shape and column names
print("Full Raw Dataset Shape:", df_raw.shape)
print("Incremental Dataset Shape:", df_incremental.shape)


Full Raw Dataset Shape: (100, 7)
Incremental Dataset Shape: (10, 7)


### Transformation 1: Cleaning

In this step, we address data quality by:
- Removing rows with critical missing values (e.g., quantity, unit_price)
- Dropping exact duplicate rows
This ensures that downstream analysis uses only valid and unique records.


In [2]:
# Work on a copy to preserve original
df_clean_full = df_raw.copy()

# Show missing values before
print("Missing Values (Before Cleaning):")
print(df_clean_full.isnull().sum())

# Drop rows where quantity or unit_price is missing
df_clean_full = df_clean_full.dropna(subset=["quantity", "unit_price"])

# Drop duplicate rows
df_clean_full = df_clean_full.drop_duplicates()

# Show shape after cleaning
print("\nAfter Cleaning - Shape:", df_clean_full.shape)

# Optional: Preview cleaned data
df_clean_full.head()


Missing Values (Before Cleaning):
order_id          0
customer_name     1
product           0
quantity         26
unit_price       35
order_date        1
region           25
dtype: int64

After Cleaning - Shape: (46, 7)


,order_id,customer_name,product,quantity,unit_price,order_date,region
2,3,Charlie,Laptop,2.0,250.0,2024-01-08,NaN
3,4,Eve,Laptop,2.0,750.0,2024-01-07,West
6,7,Charlie,Monitor,2.0,750.0,2024-02-02,West
9,10,Eve,Monitor,1.0,500.0,2024-02-28,North
10,11,NaN,Monitor,3.0,750.0,2024-04-24,West


#### Apply Cleaning to Incremental Dataset

We apply the same cleaning process (missing values and duplicates) to the incremental dataset for consistency across the pipeline.


In [3]:
# Work on a copy to preserve original
df_clean_incremental = df_incremental.copy()

# Show missing values before
print("Missing Values (Before Cleaning):")
print(df_clean_incremental.isnull().sum())

# Drop rows where quantity or unit_price is missing
df_clean_incremental = df_clean_incremental.dropna(subset=["quantity", "unit_price"])

# Drop duplicate rows
df_clean_incremental = df_clean_incremental.drop_duplicates()

# Show shape after cleaning
print("\nAfter Cleaning - Shape:", df_clean_incremental.shape)

# Optional: Preview cleaned data
df_clean_incremental.head()


Missing Values (Before Cleaning):
order_id         0
customer_name    6
product          0
quantity         4
unit_price       0
order_date       0
region           2
dtype: int64

After Cleaning - Shape: (6, 7)


,order_id,customer_name,product,quantity,unit_price,order_date,region
1,102,NaN,Laptop,1.0,300.0,2024-05-07,Central
2,103,NaN,Laptop,1.0,600.0,2024-05-04,Central
4,105,Heidi,Tablet,2.0,600.0,2024-05-21,North
5,106,NaN,Laptop,2.0,600.0,2024-05-18,Central
6,107,NaN,Tablet,1.0,600.0,2024-05-13,Central


###  Cleaning Summary

We applied missing value handling and duplicate removal to both datasets. The goal was to ensure each row had complete and reliable values in critical numeric columns (`quantity`, `unit_price`), and to remove redundant data.

####  Full Dataset
- **Before Cleaning**: 100 rows, 7 columns  
- **After Cleaning**: 46 rows, 7 columns  
- **Rows Removed**: 54 (due to missing values or duplicates)

####  Incremental Dataset
- **Before Cleaning**: 10 rows, 7 columns  
- **After Cleaning**: 6 rows, 7 columns  
- **Rows Removed**: 4 (due to missing values or duplicates)

This cleaning ensures that our transformation logic operates on complete and unique data only.


### Transformation 2: Enrichment

We add a new derived column called `total_price`, calculated as:
> total_price = quantity × unit_price

This allows future analysis such as total revenue per product or customer.


In [4]:
# Create total_price column in full dataset
df_clean_full["total_price"] = df_clean_full["quantity"] * df_clean_full["unit_price"]

# Preview the result
df_clean_full[["quantity", "unit_price", "total_price"]].head()


,quantity,unit_price,total_price
2,2.0,250.0,500.0
3,2.0,750.0,1500.0
6,2.0,750.0,1500.0
9,1.0,500.0,500.0
10,3.0,750.0,2250.0


In [5]:
# Create total_price column in incremental dataset
df_clean_incremental["total_price"] = df_clean_incremental["quantity"] * df_clean_incremental["unit_price"]

# Preview the result
df_clean_incremental[["quantity", "unit_price", "total_price"]].head()


,quantity,unit_price,total_price
1,1.0,300.0,300.0
2,1.0,600.0,600.0
4,2.0,600.0,1200.0
5,2.0,600.0,1200.0
6,1.0,600.0,600.0


####  Enrichment Summary

A new column, `total_price`, was added to both datasets using:

> `total_price = quantity * unit_price`

This column captures the total transaction value per order.

####  Full Dataset (Preview)
Before:  
| quantity | unit_price |
|----------|------------|
| 2.0      | 250.0      |
| 2.0      | 750.0      |

After:  
| quantity | unit_price | total_price |
|----------|------------|-------------|
| 2.0      | 250.0      | 500.0       |
| 2.0      | 750.0      | 1500.0       |

The same transformation was applied to the incremental dataset.


### Transformation 3: Structural Change – Standardize Date Format

To enable time-based analysis, we convert the `order_date` column in both datasets from object (string) format to proper Python datetime format using `pd.to_datetime()`.


In [ ]:
# Convert order_date to datetime format in full data
df_clean_full["order_date"] = pd.to_datetime(df_clean_full["order_date"], errors="coerce")

# Check data type and preview
print(df_clean_full["order_date"].dtypes)
df_clean_full[["order_id", "order_date"]].head()


datetime64[ns]


,order_id,order_date
2,3,2024-01-08
3,4,2024-01-07
6,7,2024-02-02
9,10,2024-02-28
10,11,2024-04-24


In [7]:
# Convert order_date to datetime format in incremental data
df_clean_incremental["order_date"] = pd.to_datetime(df_clean_incremental["order_date"], errors="coerce")

# Check data type and preview
print(df_clean_incremental["order_date"].dtypes)
df_clean_incremental[["order_id", "order_date"]].head()


datetime64[ns]


,order_id,order_date
1,102,2024-05-07
2,103,2024-05-04
4,105,2024-05-21
5,106,2024-05-18
6,107,2024-05-13


####  Date Format Summary

The `order_date` column in both datasets has been successfully converted to datetime format.  
This allows filtering by time ranges or extracting month/year for future reporting.

- Converted using: `pd.to_datetime(column)`
- Invalid or malformed dates were set to NaT (`errors="coerce"`)


### Transformation 4: Categorization – Price Tiers

We create a new categorical column `price_tier` based on the `total_price` value:

- `Low`: ≤ 100
- `Medium`: 101–500
- `High`: > 500

This can help segment customers or transactions for business insights.


In [8]:
# Define price tier function(Full dataset)
def price_tier(value):
    if value <= 100:
        return "Low"
    elif value <= 500:
        return "Medium"
    else:
        return "High"

# Apply to full dataset
df_clean_full["price_tier"] = df_clean_full["total_price"].apply(price_tier)

# Preview
df_clean_full[["total_price", "price_tier"]].head()


,total_price,price_tier
2,500.0,Medium
3,1500.0,High
6,1500.0,High
9,500.0,Medium
10,2250.0,High


In [9]:
# Apply to incremental dataset
df_clean_incremental["price_tier"] = df_clean_incremental["total_price"].apply(price_tier)

# Preview
df_clean_incremental[["total_price", "price_tier"]].head()


,total_price,price_tier
1,300.0,Medium
2,600.0,High
4,1200.0,High
5,1200.0,High
6,600.0,High


####  Categorization Summary

We added a new `price_tier` column based on `total_price` values:

| Tier      | Range       |
|-----------|-------------|
| Low       | ≤ 100       |
| Medium    | 101 – 500   |
| High      | > 500       |

This transformation helps with customer segmentation or analyzing purchase behaviors.


### Save the Full and Incremental Datasets
Both cleaned and enriched datasets are saved in the `transformed/` directory:

- `transformed/transformed_full.csv`
- `transformed/transformed_incremental.csv`

These outputs will be used in the Load phase.


In [10]:
# Save the transformed full dataset
df_clean_full.to_csv("transformed/transformed_full.csv", index=False)

# Save the transformed incremental dataset
df_clean_incremental.to_csv("transformed/transformed_incremental.csv", index=False)

print("Transformed datasets saved successfully.")


Transformed datasets saved successfully.
